In [3]:
!pip install keras-tuner

  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)
  Using cached tqdm-4.55.1-py2.py3-none-any.whl (68 kB)
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78936 sha256=9757eace8f0e1b6cca6e02242f445131a281243ad78d46522cbdd7f6c01f3de5
  Stored in directory: c:\users\virat.singh\appdata\local\pip\cache\wheels\53\3d\c3\160c686bd74a18989843fcd015e8f6954ca8d834fd2ef4658a
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15355 sha256=e866bb89efa4e5bd23b82fce5782177497b9caed7ad74c61d62150902807d1d3
  Stored in directory: c:\users\virat.singh\appdata\local\pip\cache\wheels\08\8f\5f\253d0105a55bd84ee61ef0d37dbf70421e61e0cd70cef7c5e1
Successfully built keras-tuner terminaltables


In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
data=pd.read_csv('Churn_Modelling.csv')

In [6]:
X=data.iloc[:,3:13]
y=data.iloc[:,13]

In [8]:
geography=pd.get_dummies(X['Geography'],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [9]:
X=pd.concat([X,geography,gender],axis=1)

In [10]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [11]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [13]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Churn')

In [14]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 34s]
val_accuracy: 0.7975000143051147

Best val_accuracy So Far: 0.7975000143051147
Total elapsed time: 00h 02m 05s
INFO:tensorflow:Oracle triggered exit
